# Predicting Game Newsletter Subscription Using K-Nearest Neighbors

## Introduction
The purpose of this data science project is to analyze and predict player behaviours on a Minecraft server based on data collected by a UBC Computer Science research group. This data consisted of two primary files: players.csv and sessions.csv. The players.csv file included detailed information about each player, such as their age, experience level, email, and other personal traits. The sessions.csv file tracked individual play sessions for each player based on their email, including data on session times. 



### Research Question

**What player characteristics and behaviors are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types?**

The specific question we addressed in this project is: "Can experience level, age, and total hours played predict newsletter subscription?"

By analyzing both the age and the experience level of a player, we will build a predictive model (K-NN Classification) to assess the likelihood of player subscribing to the newsletter based on such characteristics.


To answer this question, we focus solely on using the players.csv dataset file, as it includes the relevant characteristics (age and experience level) we will be using to hypothesize a player's subscription behaviour. This will involve exploring the dataset to understand the relationships between these variables and subscription status, followed by the development and assessment of the accuracy of our model to form predictions. 



### Dataset Description

The `players` dataset contains 196 observations with the following variables:

- `experience` (chr): Player experience level (Pro, Veteran, Regular, Amateur)
- `subscribe` (lgl): Whether the player subscribed to the newsletter
- `played_hours` (dbl): Total hours played
- `Age` (dbl): Player's age
- `gender` (chr): Player's gender
- `hashedEmail` (chr): Player's email (not used in this analysis)
- `name` (chr): Player's name (not used in this analysis)

This analysis uses techniques covered in the DSCI 100 course including data wrangling, visualization, and K-Nearest Neighbors classification.


## Methods and Results

K-NN classification is a method where the goal is to predict a categorical label. In the case of this project, the K-NN classification model will work well in helping classify whether a player is or will be suscribed to the newsletter based on the player's features.

K-NN classifies new observations by comparing them to similar, labeled examples in the training set. This is effective when we want to base predictions not on assumptions about the data, but on how similar a new observation is to other known observations. Since we are working with a binary classification (TRUE/FALSE) and want to leverage patterns found in past player behavior, K-NN is a suitable choice for building our predictive model.







**We selected the K-Nearest Neighbors (KNN) algorithm because it is non-parametric, easy to understand, and directly influenced by the similarity between features. This cooperates well with our small dataset and helps build an easy understanding of how different features impact prediction outcomes.**

**Because our predictors are categorical variables, we performed one-hot encoding to convert them into a numerical format to be used for the KNN algorithm.**

**Before modeling, we prepared the dataset as follows:**

- Removed unnecessary variables (ie `name`, `hashedEmail`)
- Removed rows with missing values using na.omit()



In [ ]:
# load the necessary libraries and packages
library(tidyverse)
library(class)
library(ggplot2)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 10)
library(recipes)
library(themis)
library(dplyr)

In [ ]:
# Reading players.csv file

players <- read_csv("players.csv") 

players_clean <- players |>
    select(-hashedEmail, -name) |>
    mutate(subscribe = as.factor(subscribe)) |>
    na.omit()

# view the first 6 rows of data
head(players_clean)

Upon inspecting the file, we observed that the number of subscribers exceeds the number of non-subscribers. To verify this, we created a table to compare the counts of each group. As shown below, 73% are subscribers, while 27% are non-subsribers. Given this class imbalance, we will upsample the response variable once we build the model to ensure a more balanced representation.

In [ ]:
num_obs <- nrow(players)

players_subscribe <- players |>
  group_by(subscribe) |>
  summarize(
    count = n(),
    percentage = n() / num_obs * 100
  )

players_subscribe

We then summarized numeric features to understand their scale and variance (e.g., max, min, mean).

In [ ]:
players_clean |> 
  summarise(
    max_hours = max(played_hours),
    min_hours = min(played_hours),
    mean_hours = mean(played_hours),
    max_age = max(Age),
    min_age = min(Age),
    mean_age = mean(Age)
  )


### **Findings on Player Type Differences**  

We visualized three relationships:

- Subscription Rate by Experience Level

- Played Hours vs Subscription

- Age Distribution vs Subscription


Based on the visualizations generated, we can analyze the vary in characteristics and how these factors influence **subscription behaviour**.

**Figure 1** below illustrates subscription rate by experience level.

In [ ]:
ggplot(players, aes(x = experience, fill = subscribe)) +
  geom_bar(position = "dodge") +
  labs(title = "Figure 1. Subscription Rate by Experience Level", x = "Experience", y = "Count", fill = "Subscribed")


**Figure 2** illsutrates the relationship between played hours and subscription below.

In [ ]:
ggplot(players, aes(x = subscribe, y = played_hours, color = subscribe)) +
  geom_point() +
  labs(
    title = "Figure 2. Played Hours vs Subscription",
    x = "Subscribed",
    y = "Played Hours"
  )


**Figure 3** below illustrates the dis

In [ ]:
ggplot(players, aes(x = Age, fill = subscribe)) +
  geom_histogram(bins = 15, alpha = 0.6, position = "identity", color = "black") +
  labs(title = "Figure 3. Age Distribution by Subscription", x = "Age", y = "Count", fill = "Subscribed")

In [ ]:
# ggplot(players, aes(x = gender, fill = factor(subscribe))) +
#   geom_bar(position = "fill") +
#   labs(
#     title = "Figure 4. Subscription Proportion by Gender",
#     x = "Gender",
#     y = "Proportion",
#     fill = "Subscribed"
#   )


#### **1 Experience Level vs Subscription Rate**
- Pro and Veteran players have the highest subscription rates, while Regular and Amateur players have lower subscription rates.

#### **2 Played Hours vs Subscription**
- Subscribed players tend to have higher median played hours.


#### **3 Age vs Subscription**
- Younger players are more likely to subscribe, at same time older players (above 30) show a lower subscription rate.

- Older players may focus on the playing alone


### **Training the Model (edit this later)**  

We first split the dataset into training and test sets using an 70/30 ratio using `initial_split()` function. 

In [ ]:
set.seed(123)

players_split <- initial_split(players_clean, prop = 0.70, strata = subscribe)

players_train <- training(players_split)

players_test <- testing(players_split)

Then we create the recipe, ensuring to use the `scale()` function to normalize numerical predictors to ensure they are on comparable scales, which is essential for K-NN performance. 

As mentioned in the introduction, we also perform one-hot encoding through the `step_dummmy()` function to convert `experience` into numeric binary terms. Due to class imbalance (fewer non-subscribers than subscribers), we upsample the subscribe variable using `step_upsample()`. This will create a balanced dataset by replicating non-subscriber cases until both have equal representation. 

Finally, we define a K-NN model with `nearest_neighbor()` adding the defined recipe and with an initial guess of 3 neighbours which will later be tuned.

In [ ]:
set.seed(3030) # DO NOT REMOVEz
players_recipe <- recipe(subscribe ~ Age + experience + played_hours, data = players_clean) |>
    step_dummy(experience, one_hot = TRUE) |> # To turn into numeric binary model terms
    step_upsample(subscribe, over_ratio = 1) |>  # To upsample subscribers : over_ratio = 1 means 1:1 ratio
    step_scale(all_predictors()) |>
    step_center(all_predictors()) 

players_recipe

knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 3) |>
    set_engine("kknn") |>
    set_mode("classification")


To find the best k value, we can evaluate the accuracy of each k value by performing a 10-fold cross-validation split of our training data using `vfold_cv()` setting strata as our categorical label variable, `subscribe`. The new workflow we create is fit across all cross-validation folds through `fit_resamples()`, and we collect the aggregated performance metrics, quantifying how well our model performs across multiple validation runs.

Making a new K-NN specification, we make the number of neighbours as tunable through `tune()`, then create a grid of potential k values ranging from 1 to 40. Using tune_grid(), we evaluate each k value with our 10-fold cross-validation. After collecting all results, we filter the accuracy metrics and visualize the relationship between k and model accuracy, and the result was plotted as Figure 5, showing how model performance varies with K. This helps identify the value where accuracy peaks before overfitting or underfitting potentially occurs.

We then select the value of 26 as the best k value.

In [ ]:
set.seed(1010) # DO NOT REMOVE


players_vfold <- vfold_cv(players_train, v = 10, strata = subscribe)

players_vfold_fit <- workflow() |>
    add_recipe(players_recipe) |>
    add_model(knn_spec) |>
    fit_resamples(resamples = players_vfold)


players_metrics <- collect_metrics(players_vfold_fit)

knn_tune <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
    set_engine("kknn") |>
    set_mode("classification")

k_vals <- tibble(neighbors = seq(from = 1, to = 40, by = 1))

knn_results <- workflow() |>
    add_recipe(players_recipe) |>
    add_model(knn_tune) |>
    tune_grid(resamples = players_vfold, grid = k_vals) |>
    collect_metrics()

accuracies <- knn_results |>
  filter(.metric == "accuracy")

accuracy_vs_k <- ggplot(accuracies, aes(x = neighbors, y = mean)) +
  geom_point() +
  geom_line(color = "blue") +
  labs(
    title = "Figure 5. KNN Accuracy vs K",
    x = "Number of Neighbors (K)",
    y = "Prediction Accuracy"
  ) +
  theme(text = element_text(size = 12))

accuracy_vs_k

best_k <- accuracies |>
        arrange(desc(mean)) |>
        head(1) |>
        pull(neighbors)
best_k

After identifying the optimal k value from above (`best_k`), we create a new K-NN specification model using this value. The model was combined with the previously preprocessed recipe in a new workflow, which was then fit to our entire training set (`players_train`) to create the final predictive model. 

In [ ]:
set.seed(4040) # DO NOT REMOVE

# Creating model
knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = best_k) |>
  set_engine("kknn") |>
  set_mode("classification")

knn_fit <- workflow() |>
  add_recipe(players_recipe) |>
  add_model(knn_spec) |>
  fit(data = players_train)


Using our model, we generate predictions on our test set (`players_test`) to evaluate its performance on unseen data.

In [ ]:
set.seed(2020) # DO NOT REMOVE

players_test_predictions <- predict(knn_fit, players_test) |>
  bind_cols(players_test)

#players_test_predictions 


Finally, we calculated the accuracy of our model by comparing it with the estimate of the response variable to the actual observation. We first compute the overall classification accuracy, which represents the proportion of correct predictions across all observations. We then examine precision, which measures how many of the predicted positive cases were actually positive. Then, we assess recall , which indicates shows what proportion of actual positive cases our model successfully identified.

In [ ]:
set.seed(2020) # DO NOT REMOVE

players_test_predictions |>
  metrics(truth = subscribe, estimate = .pred_class) |>
  filter(.metric == "accuracy")

players_test_predictions |>
    precision(truth = subscribe, estimate = .pred_class, event_level="first")

players_test_predictions |>
    recall(truth = subscribe, estimate = .pred_class, event_level="first")

Given our model's suboptimal metrics (71% accuracy, 45% precision, and 31% recall), we conduct a deeper  examination using a confusion matrix to understand the specific patterns in prediction errors.

In [ ]:
confusion <- players_test_predictions |>
             conf_mat(truth = subscribe, estimate = .pred_class)
confusion

## Discussion

### What we found
**Figure 1: Subscription Rate by Experience Level**

Analysis:
The plot clearly shows that players with higher experience levels (Veteran and Pro) have higher subscription rates than Regular or Amateur players. This supports the assumption that more experienced players are more engaged with the game and thus more likely to subscribe. What's more, it may also reflect that experienced players have been in the game longer and had more possibility to see the newsletter option.

 **Figure 2: Played Hours vs Subscription (Boxplot)**

Analysis :
The boxplot indicates that subscribed players tend to have higher median and maximum playtime. This seems intuitive—more invested players may want to stay updated. However, it’s also possible that subscription leads to more engagement over time (reverse causality)

**Figure 3: Age Distribution by Subscription**

Analysis :
Younger players appear more likely to subscribe. This may reflect digital habits: younger users tend to be more open to email subscriptions, and more accustomed to in-game notifications and marketing. Oppoesitely, it may be that older users are less willing o accpect                                                                 

**Predictive Model (Edit Later)**


Analysis:
Some thoughts I have are
- I have a feeling that the data is self-reported. I caught instances where someone would have an experience level of Veteran but have 0 played hours... there's definitely errors in the data which would lead to errors in predictions. If I knew how to do this, I would probs have it so that it would filter let's say Veterans and in order for them to stay in the data set they would need maybe 100 hours or more. Otherwise, omit the observation
- Age, played_hours, and experience level are just overall poor predictors and aren't causal factors but more corellational. Like coincidentally let's say older players just subscribed more but there's no meaning behind that, or maybe players who are subscribed are just willing to play more because they are subscribed (reverse causality). These features aren't really an influence or factor that would lead them to subscribe if that makes sense
- I feel like better predictors could be rather than like a player's personal features, maybe how they interact with the game would have been better. Like how much someone pays for the game like in-game purchases, or maybe if they join gaming tournaments. Also, since we didn't use sessions.csv maybe played_sessions would have been better predictors.



These findings can help game developers better understand user behavior. For example, promotional efforts could be tailored to high-playtime players to increase conversions. Knowing that experience and age influence subscription likelihood, marketing emails could highlight tips, achievements, or community events to attract younger or more engaged players.Furthermore,The game team could also consider promoting newsletter subscriptions through platforms like Instagram and TikTok, which are more popular among younger audiences.

## Project Limitations and Improvements
**Sample Size**: With only 196 players in the dataset, the sample may not be sufficiently representative of the overall player population. In particular, certain groups of people (e.g., older players, non-male genders, or very casual users) may be underrepresented, potentially biasing the model’s conclusions. Expanding the dataset with more diverse and balanced samples would improve the robustness and generalizability of the findings.

**Feature Limitation**: The model currently relies on a limited set of variables: age, gender, experience, and total playtime. These features, while relevant, provide only a static snapshot of user behavior. Incorporating dynamic behavioral indicators such as login frequency, session duration, recent activity, or interaction with in-game content could significantly enhance predictive power. Behavioral features often capture intent and engagement more effectively than static demographics.

**Platform Bias**: The dataset does not capture from which channels players encountered or accessed the subscription option. Players who engage primarily via mobile or social platforms might behave differently. Future data collection could include interaction channels to analyze cross-platform behavior differences.